In [1]:

# import findspark
# findspark.init()
from  pyspark.sql.functions import input_file_name
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ClusterAnalysis").config('spark.sql.shuffle.partitions','10').getOrCreate()



from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the data stored in LIBSVM format as a DataFrame.
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
# summary only
print(treeModel)



21/12/12 21:08:52 WARN Utils: Your hostname, daniyal resolves to a loopback address: 127.0.1.1; using 192.168.9.38 instead (on interface wlp1s0)
21/12/12 21:08:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/12 21:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/12 21:08:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/12 21:08:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/12 21:08:56 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[100,101,102...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[126,127,128...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.0333333 
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4e8e8921020a, depth=2, numNodes=5, numClasses=2, numFeatures=692


In [3]:
!pip install py4j

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install spark-submit

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for spark-submit: filename=spark_submit-1.2.0-py3-none-any.whl size=9396 sha256=6d7d88bf0dd32aa9bf27ed22681dbd64f6140fb3eafa2484c13021f54abacf6f
  Stored in directory: /home/daniyal/.cache/pip/wheels/94/a7/49/a04ca922bf65d424bd99523eeff360ebbc82c297f387df14b3
Successfully built spark-submit
